# Total de actores/actrices candidatos al Óscar que tienen las películas más famosas por género.

## Y al final contestar dos preguntas:

* ¿Cuál es el género con mayor número de candidaturas?

* ¿Cuál es la película con mayor diferencia porcentual de candidaturas respecto a la media de su género?

In [1]:
import re
import json
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Extraer las 30 mejores películas de los geneos de acción, comédia, drama y terror en FilmAffinity, además películas del año 1980 en adelante.

In [2]:
full_peliculas = []  # Lista vacía para almacenar la información de las películas
generos = {'acción', 'comedia', 'drama', 'terror'}  # Conjunto de géneros

# Iterar sobre cada género
for genero in generos:
    # Construir la URL para cada género
    genre_url = f'https://www.filmaffinity.com/es/topgen.php?genres=%2B{genero[:2].upper()}&chv=0&orderby=rc&movietype=movie%7C&country=&fromyear=1980&toyear=2023&ratingcount=2&runtimemin=0&runtimemax=4'
    
    # Realizar una solicitud GET a la URL del género y obtener la página
    req = requests.get(genre_url)
    
    # Crear un objeto BeautifulSoup para analizar el HTML de la página del género
    soup = BeautifulSoup(req.text, 'html.parser')
    
    # Encontrar todas las películas en la página del género utilizando la clase 'mc-title'
    for pelicula in soup.select('.mc-title'):
        # Obtener el título de la película
        titulo = pelicula.text.strip()
        
        # Encontrar el div siguiente que contiene el reparto utilizando la clase 'mc-cast'
        cast_div = pelicula.find_next_sibling(class_='mc-cast')
        
        # Obtener los nombres de los actores de los elementos '.nb' dentro del div de reparto
        actores = [actor.text.strip() for actor in cast_div.select('.nb') if actor.a]
        
        # Agregar la información de la película a la lista de películas
        full_peliculas.append({
            "title": titulo.split("(")[0].strip(),  # Título de la película
            "year": titulo.split("(")[1].strip().replace(")", ""),  # Año de la película
            "genre": genero,  # Género de la película
            "actors": ", ".join(actores)  # Lista de actores de la película
        })    

# Devolver la lista completa de películas recolectadas
full_peliculas


[{'title': 'Forrest Gump',
  'year': '1994',
  'genre': 'comedia',
  'actors': 'Tom Hanks,, Robin Wright,, Gary Sinise,, Mykelti Williamson,, Sally Field,, Rebecca Williams,, Michael Conner Humphreys,, Haley Joel Osment'},
 {'title': 'La vida es bella',
  'year': '1997',
  'genre': 'comedia',
  'actors': 'Roberto Benigni,, Nicoletta Braschi,, Giorgio Cantarini,, Marisa Paredes,, Giustino Durano,, Horst Buchholz,, Sergio Bini Bustric'},
 {'title': 'American Beauty',
  'year': '1999',
  'genre': 'comedia',
  'actors': 'Kevin Spacey,, Annette Bening,, Thora Birch,, Allison Janney,, Peter Gallagher,, Mena Suvari,, Wes Bentley,, Chris Cooper'},
 {'title': 'El rey león',
  'year': '1994',
  'genre': 'comedia',
  'actors': 'Matthew Broderick,, Jeremy Irons,, Jonathan Taylor Thomas,, Moira Kelly,, Rowan Atkinson,, Whoopi Goldberg,, Cheech Marin'},
 {'title': 'Toy Story',
  'year': '1995',
  'genre': 'comedia',
  'actors': 'Tom Hanks,, Tim Allen,, Don Rickles,, Jim Varney,, Wallace Shawn,, John

In [3]:
movies = pd.DataFrame(full_peliculas)
movies["name"] = movies["actors"].str.split(",")
movies

,title,year,genre,actors,name
0,Forrest Gump,1994,comedia,"Tom Hanks,, Robin Wright,, Gary Sinise,, Mykel...","[Tom Hanks, , Robin Wright, , Gary Sinise, ,..."
1,La vida es bella,1997,comedia,"Roberto Benigni,, Nicoletta Braschi,, Giorgio ...","[Roberto Benigni, , Nicoletta Braschi, , Gio..."
2,American Beauty,1999,comedia,"Kevin Spacey,, Annette Bening,, Thora Birch,, ...","[Kevin Spacey, , Annette Bening, , Thora Bir..."
3,El rey león,1994,comedia,"Matthew Broderick,, Jeremy Irons,, Jonathan Ta...","[Matthew Broderick, , Jeremy Irons, , Jonath..."
4,Toy Story,1995,comedia,"Tom Hanks,, Tim Allen,, Don Rickles,, Jim Varn...","[Tom Hanks, , Tim Allen, , Don Rickles, , J..."
...,...,...,...,...,...
115,American Psycho,2000,terror,"Christian Bale,, Willem Dafoe,, Jared Leto,, J...","[Christian Bale, , Willem Dafoe, , Jared Let..."
116,Abierto hasta el amanecer,1996,terror,"George Clooney,, Harvey Keitel,, Juliette Lewi...","[George Clooney, , Harvey Keitel, , Juliette..."
117,Destino final,2000,terror,"Devon Sawa,, Ali Larter,, Kerr Smith,, Seann W...","[Devon Sawa, , Ali Larter, , Kerr Smith, , ..."
118,Déjame entrar,2008,terror,"Kåre Hedebrant,, Lina Leandersson,, Per Ragnar...","[Kåre Hedebrant, , Lina Leandersson, , Per R..."


In [4]:
movies = movies.explode("name").drop(columns="actors")
movies["name"] = movies["name"].str.strip()

In [5]:
movies = movies.reset_index(drop=True)

In [6]:
movies

,title,year,genre,name
0,Forrest Gump,1994,comedia,Tom Hanks
1,Forrest Gump,1994,comedia,
2,Forrest Gump,1994,comedia,Robin Wright
3,Forrest Gump,1994,comedia,
4,Forrest Gump,1994,comedia,Gary Sinise
...,...,...,...,...
1742,Death Proof,2007,terror,Rose McGowan
1743,Death Proof,2007,terror,
1744,Death Proof,2007,terror,Tracie Thoms
1745,Death Proof,2007,terror,


## Extraer los galárdones del óscar a partir del año de 1980. 

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

actor_types = [
    ('al', 'actor'),
    ('a_la', 'actriz'),
    ('al', 'actor_de_reparto'),
    ('a_la', 'actriz_de_reparto')
]

all_actors_movies = []

for art, actor_type in actor_types:
    url = f'https://es.wikipedia.org/wiki/Anexo:%C3%93scar_{art}_mejor_{actor_type}'

    # Realizar la solicitud GET a la página
    response = requests.get(url)

    # Crear el objeto BeautifulSoup para analizar el HTML
    soup = BeautifulSoup(response.content, 'html.parser')

    # Encontrar todas las tablas en la página
    tablas = soup.find_all('table')

    # Lista para almacenar los actores y películas de esta URL
    actors_movies = []

    # Iterar sobre cada tabla y extraer los nombres de los actores y películas
    for tabla in tablas:
        filas = tabla.find_all('tr')  # Encontrar todas las filas de la tabla
        for fila in filas:
            columnas = fila.find_all(['th', 'td'])  # Encontrar las columnas (celdas de encabezado y datos)
            datos_fila = [columna.text.strip() for columna in columnas]  # Extraer texto de cada celda
            # Añadir a la lista si hay datos en la fila y no es un encabezado
            if datos_fila and len(datos_fila) >= 4 and datos_fila[0].isdigit():
                actor_pelicula = {
                    'actor': datos_fila[2],
                    'pelicula': datos_fila[3]
                }
                actors_movies.append(actor_pelicula)

    # Añadir los actores y películas extraídos de esta URL a la lista principal
    all_actors_movies.append({
        'actor_type': actor_type,
        'actors_movies': actors_movies
    })

# Crear un DataFrame con los datos recolectados
data = []
for table_info in all_actors_movies:
    for actor_pelicula in table_info['actors_movies']:
        data.append([actor_pelicula['actor'], actor_pelicula['pelicula']])

oscars = pd.DataFrame(data, columns=['name', 'Película'])

oscars

,name,Película
0,Brendan Fraser,The Whale
1,Will Smith,King Richard
2,Anthony Hopkins,The Father
3,Joaquin Phoenix,Joker
4,Rami Malek,Bohemian Rhapsody
...,...,...
438,Jane Darwell,The Grapes of Wrath
439,Hattie McDaniel,Lo que el viento se llevó
440,Fay Bainter,Jezabel
441,Alice Brady,In Old Chicago


In [8]:
oscars = oscars.groupby("name").size().sort_values()
oscars = oscars.reset_index()
oscars.columns = ["name", "nominations"]
oscars

,name,nominations
0,Adrien Brody,1
1,Marcia Gay Harden,1
2,Lupita Nyong'o[1]​[2]​[3]​,1
3,Louise Fletcher,1
4,"Louis Gossett, Jr.",1
...,...,...
308,Frances McDormand,3
309,Daniel Day-Lewis,3
310,Ingrid Bergman,3
311,Katharine Hepburn,4


In [9]:
df = movies.merge(oscars, on="name", how='inner')
df

,title,year,genre,name,nominations
0,Forrest Gump,1994,comedia,Tom Hanks,2
1,Toy Story,1995,comedia,Tom Hanks,2
2,Toy Story 3,2010,comedia,Tom Hanks,2
3,Toy Story 2,1999,comedia,Tom Hanks,2
4,Forrest Gump,1994,drama,Tom Hanks,2
...,...,...,...,...,...
111,Réquiem por un sueño,2000,drama,Ellen Burstyn,1
112,Soy leyenda,2007,terror,Emma Thompson,1
113,Drácula de Bram Stoker,1992,terror,Anthony Hopkins,2
114,Bienvenidos a Zombieland,2009,terror,Emma Stone,1


In [10]:
df.to_json("movies_actors.json")

## Género con más nominaciones:

In [11]:
df = pd.read_json("movies_actors.json")

In [12]:
df.groupby("genre")["nominations"].sum().sort_values(ascending=False)

genre
drama      55
comedia    39
acción     37
terror     16
Name: nominations, dtype: int64

## El género con más nominaciones es el drama.

In [13]:
df = df.groupby(["title", "genre"])["nominations"].sum()

In [14]:
df = df.reset_index()
df

,title,genre,nominations
0,Abierto hasta el amanecer,terror,1
1,American Beauty,comedia,4
2,American Beauty,drama,4
3,American Psycho,terror,2
4,Batman,acción,5
...,...,...,...
58,Toy Story 2,comedia,2
59,Toy Story 3,comedia,2
60,Up,comedia,1
61,V de Vendetta,acción,1


In [15]:
#Media por nominación por cada género
df.groupby("genre")["nominations"].mean()

genre
acción     2.466667
comedia    2.052632
drama      2.894737
terror     1.600000
Name: nominations, dtype: float64

In [16]:
#ratio de nominación por cada película.
df["ratio_nominations"] = df.groupby("genre")["nominations"].transform(lambda x: x/x.mean())
df

,title,genre,nominations,ratio_nominations
0,Abierto hasta el amanecer,terror,1,0.625000
1,American Beauty,comedia,4,1.948718
2,American Beauty,drama,4,1.381818
3,American Psycho,terror,2,1.250000
4,Batman,acción,5,2.027027
...,...,...,...,...
58,Toy Story 2,comedia,2,0.974359
59,Toy Story 3,comedia,2,0.974359
60,Up,comedia,1,0.487179
61,V de Vendetta,acción,1,0.405405


In [17]:
df.nlargest(3, columns="ratio_nominations")

,title,genre,nominations,ratio_nominations
38,Mejor... imposible,comedia,5,2.435897
11,El caballero oscuro,acción,6,2.432432
12,El caballero oscuro,drama,6,2.072727


In [27]:
movies[movies['title']=='Mejor... imposible']

,title,year,genre,name
222,Mejor... imposible,1997,comedia,Jack Nicholson
223,Mejor... imposible,1997,comedia,
224,Mejor... imposible,1997,comedia,Helen Hunt
225,Mejor... imposible,1997,comedia,
226,Mejor... imposible,1997,comedia,Greg Kinnear
227,Mejor... imposible,1997,comedia,
228,Mejor... imposible,1997,comedia,Cuba Gooding Jr.
229,Mejor... imposible,1997,comedia,
230,Mejor... imposible,1997,comedia,Skeet Ulrich
231,Mejor... imposible,1997,comedia,


## Película con mayor diferencia porcentual respecto a la media de su género es Mejor... Imposible (1997) protagonizada por Jack Nocholson.

In [18]:
df.nsmallest(3, columns="ratio_nominations")

,title,genre,nominations,ratio_nominations
17,El pianista,drama,1,0.345455
32,La lista de Schindler,drama,1,0.345455
34,La vida es bella,drama,1,0.345455


## Películas con menores porcentuales respecto a la media de su género es El Pianista, La lista de Schindler y La vida es bella.